In [1]:
#Se importa la libreria pandas para trabajar
import pandas as pd

In [2]:
#Se crea una lista con años para luego utilizar a la hora de leer los archivos.

añosProyectos = ['2008',
                 '2009',
                 '2010',
                 '2011',
                 '2012',
                 '2013',
                 '2014',
                 '2015',
                 '2016',
                 '2017',
                 '2018',
                 '2019']

In [3]:
#Se inicializa un dataframe vacio. Luego se leen los diferentes archivos de los proyectos de los distintos años y van agregando al dataframe
df = pd.DataFrame()
for año in añosProyectos:
    archivo = pd.read_csv(f'proyectos/proyectos_{año}.csv', delimiter=";", on_bad_lines='skip')
    df = pd.concat([df, archivo])

In [4]:
#Se leen los distintos archivos ubicados en la carpeta referencias y se crean los dataframes para luego poder trabajar
monedas = pd.read_csv('referencias/ref_moneda.csv', delimiter=';')
estadoProyecto = pd.read_csv('referencias/ref_estado_proyecto.csv', delimiter=';')
organizaciones = pd.read_csv('referencias/organizaciones.csv', delimiter= ';')
tipoProyecto = pd.read_csv('referencias/ref_tipo_proyecto.csv', delimiter=";", on_bad_lines='warn')
proyBeneficiario = pd.read_csv('referencias/proyecto_beneficiario.csv', delimiter= ';')
proyParticipante = pd.read_csv('referencias/proyecto_participante.csv', delimiter=';')
proyDisciplina = pd.read_csv('referencias/proyecto_disciplina.csv', delimiter=';')
tipoInstitucion = pd.read_csv('referencias/ref_tipo_institucion.csv', delimiter=';')
personas = pd.read_csv('referencias/personas.csv', delimiter=';', low_memory=False)
funcion = pd.read_csv('referencias/ref_funcion.csv', delimiter=';')
disciplina = pd.read_csv('referencias/ref_disciplina.csv', delimiter=';')
tipoEntidad = pd.read_csv('referencias/ref_tipo_entidad_privada.csv', delimiter=';')
estadoOrganizacion = pd.read_csv('referencias/ref_estado_organizacion.csv', delimiter=';')

In [5]:
#Se extrae unicamente la fecha de inicio y finalización y se descarta la hora
fechaInicio = df['fecha_inicio'].str.split(' ', n=1, expand=True)
df['fecha_inicio'] = fechaInicio[0]
fechaFinalizacion = df['fecha_finalizacion'].str.split(' ', n=1, expand=True)
df['tipo_proyecto_id'] = df['tipo_proyecto_id'].astype('Int64')
df['fecha_finalizacion'] = fechaFinalizacion[0]

## Se trabaja con el df __monedas__

El df general tiene una columna 'moneda_id'. Con el df 'monedas' se a buscan los valores correspondientes y se reemplazan para utilizar una sola tabla y reducir una relación

In [6]:
df['tipo_proyecto_id'] = df['tipo_proyecto_id'].fillna(0)

In [7]:
#Se convierte el tipo de dato de la columna a int64 para que coincida con los otros dataframe que se utilizan.
df['tipo_proyecto_id'] = df['tipo_proyecto_id'].astype('int64')

In [8]:
#Se reemplazan los valores nulos por el dato 'Sin espeficicar'
df['fondo_anpcyt'] = df['fondo_anpcyt'].fillna('Sin especificar')

In [9]:
#Se cambian los valores de los id de las monedas por sus nombres
df['moneda_id'] = df['moneda_id'].replace(
    {1:'Pesos',
     2:'Dolares',
     3:'Euros',
     4:'Libras'
    })

## Se trabaja con el df __estadoProyecto__

El df general tiene una columna 'estado_id'. Con el df 'estadoProyecto' se a buscan los valores correspondientes y se reemplazan para utilizar una sola tabla y reducir una relación

In [10]:
#Se cambia el valor del id del estado del proyecto por su tipo de valor
df['estado_id'] = df['estado_id'].replace(
    {1:'Finalizado',
     2:'En ejecución',
     3:'Interrumpido',
     4:'No válido'
    })

In [11]:
df.head(n=1)

,proyecto_id,proyecto_fuente,titulo,fecha_inicio,fecha_finalizacion,resumen,moneda_id,monto_total_solicitado,monto_total_adjudicado,monto_financiado_solicitado,monto_financiado_adjudicado,tipo_proyecto_id,codigo_identificacion,palabras_clave,estado_id,fondo_anpcyt,cantidad_miembros_F,cantidad_miembros_M,sexo_director
0,8051,ANPCYT,Desarrollo a escala piloto de liposomas de ver...,2008/11/05,2012/12/21,No publicable,Pesos,384996.5,464959.37,190318.85,190318.65,6,ANR 600 CB 001/06,NaN,Finalizado,FONTAR,NaN,NaN,M


## Se trabaja con el archivo __tipoProyecto__
Al archivo tipo de proyecto hay que agregarle una fila para el id = 0. Esto se hace porque en el df general que vamos a trabajar hay algunos proyectos que tienen ese id. El id tendra como descripcion 'sin especificar' en todos sus valores. Esto se hace para que no queden valores vacios

Esto se hace para luego poder tener los mismos ID a la hora de trabajar los archivos y poder establecer las relaciones entre los archivos por esa columna

In [12]:
#Se crea la fila que vamos a agregar
fila_id_cero = {
    'id':0,
    'sigla':'Sin especificar',
    'descripcion':'Sin especificar',
    'tipo_proyecto_cyt_id':'Sin especificar',
    'tipo_proyecto_cyt_desc':'Sin especificar'
    }

In [13]:
#Se agrega al df de tipoProyecto
tipoProyecto = tipoProyecto._append(fila_id_cero, ignore_index=True)

#Se ordenan los valores por tipoProyecto y luego se resetan los index.
tipoProyecto = tipoProyecto.sort_values(by='id').reset_index(drop=True)

In [14]:
tipoProyecto

,id,sigla,descripcion,tipo_proyecto_cyt_id,tipo_proyecto_cyt_desc
0,0,Sin especificar,Sin especificar,Sin especificar,Sin especificar
1,1,ANR,APORTES NO REEMBOLSABLES,2,Tecnología e Innovación
2,2,ANR BIO NANO TIC,APORTES NO REEMBOLSABLES BIO NANO TIC,2,Tecnología e Innovación
3,3,ANR EMPRESA JOVEN,APORTES NO REEMBOLSABLES EMPRESA JOVEN,2,Tecnología e Innovación
4,4,ANR INT,APORTES NO REEMBOLSABLES INTERNACIONAL,2,Tecnología e Innovación
5,5,ANR P+L,APORTES NO REEMBOLSABLES PRODUCCIÓN MÁS LIMPIA,2,Tecnología e Innovación
6,6,ANR PDT,APORTES NO REEMBOLSABLES DESARROLLO TECNOLÓGICO,2,Tecnología e Innovación
7,7,AR SET,APORTES REEMBOLSABLES PARA LA PRESTACIÓN Y CON...,2,Tecnología e Innovación
8,8,ARTÍCULO 2do,ART. 2do del reglamento de beneficios promocio...,2,Tecnología e Innovación
9,9,CAE,CREDITOS A EMPRESAS,2,Tecnología e Innovación


## Se trabaja con los archivo __proyParticipante__ , __personas__ y __funcion__  

- Proyparticipante:
    - se conecta al df general por medio de 'proyecto_id'
    - se conecta al archivo de personas por 'personas_id'
    - se conecta al archivo funcion por 'funcion_id'

Se transforma el archivo proyParticipante reemplazando los valores de 'función_id' con los del archivo funcion. Esto se hace para reducir una relacion y poder obtener todo desde proyParticipante.

Lo mismo se hace con 'personas_id'. Se va a agregar una columna con los nombres a 'proyParticipante' para reducir una relación


In [15]:
proyParticipante.shape

(130429, 5)

In [16]:
funcion

,funcion_id,funcion_desc
0,1,Director
1,2,Investigador
2,3,Becario de i+d
3,4,Estudiante
4,5,Personal técnico de apoyo
5,6,Co-Director
6,7,Extensionista
7,8,Personal técnico de extensión


In [17]:
#Se crea un diccionario con los id y valores del archivo funcion. Con este diccionario se van a reemplazar luego los valores del archivo de proyParticipantes
diccionarioFuncion = {}
for index, row in funcion.iterrows():
    diccionarioFuncion[row['funcion_id']] = row['funcion_desc']

In [18]:
#Se reemplan los valores con el diccionario.
proyParticipante['funcion_id'] = proyParticipante['funcion_id'].replace(diccionarioFuncion)

In [19]:
personas

,persona_id,nombre,apellido,sexo_id,edad,cvar_ultimo_acceso
0,1,JUAN PABLO,SOTO,2.0,44.0,2017/11/29
1,2,SILVINA,GONZALEZ,1.0,36.0,2012/02/06
2,3,DIEGO FERNANDO,ASENSIO,2.0,46.0,2018/09/20
3,4,MARÍA VICTORIA,TIGNINO,1.0,37.0,2019/04/24
4,5,CLAUDIA LEDA,MATTEO,1.0,55.0,2018/09/02
...,...,...,...,...,...,...
185613,185613,CINTIA AYELEN,MARTÍNEZ,1.0,30.0,2019/07/10
185614,185614,SOPHIA,HEREDIA,1.0,29.0,2019/07/09
185615,185615,IVANNA FIORELLA,GIGLIESI,1.0,27.0,2019/07/09
185616,185616,MARIA,CERROTTA,1.0,22.0,2019/07/12


In [20]:
#Se convierten los valores de las columnas 'nombre' y 'apellido' a STR para asegurarse que se puedan concatenar
#Esto se hace para luego crear una columna que contenga los nombre y apellido para agregarla al dataframe de proyParticipante
personas['nombre'] = personas['nombre'].astype(str)
personas['apellido'] = personas['apellido'].astype(str)

In [21]:
#Se crea la columna Nombre completo
personas['Nombre completo'] = personas[['nombre', 'apellido']].agg(' '.join, axis=1)

In [22]:
#El archivo personas tenía un valor de texto. Se elimina esa fila para que no tire error al convertir los datos a int64.
personas = personas.drop(184049)

In [23]:
#Se convierten los valores a int64 para que sean iguales a los de proyParticipante y se puedan concatenar
personas['persona_id'] = personas['persona_id'].astype('int64')

In [24]:
#Se combinan los df para agregar a proyParticipante la columna Nombre completo
proyParticipante = pd.merge(proyParticipante,
                            personas[['persona_id', 'Nombre completo']],
                            on='persona_id',
                            how='left')

In [25]:
#Algunos nombre tienen un 'nan ' al principio porque había campos vacios de la tabla. Se borran eso valores para que solo quede el nombre
proyParticipante['Nombre completo'] = proyParticipante['Nombre completo'].str.replace('nan ', '')

In [26]:
proyParticipante

,proyecto_id,persona_id,funcion_id,fecha_inicio,fecha_fin,Nombre completo
0,13568,60481,Director,2012/01/01,NaN,LUIS ALEJANDRO MENDOZA ZÉLIS
1,13568,559,Investigador,2012/01/01,2012/11/14,SERGIO FERRARI
2,13568,27973,Investigador,2012/01/01,NaN,MARCOS MEYER
3,13568,36641,Investigador,2012/01/01,NaN,LAURA DAMONTE
4,13568,65908,Becario de i+d,2014/11/27,NaN,ANDRÉS TOBÍAS BIASETTI
...,...,...,...,...,...,...
130424,20786,118464,Director,2016/11/30,NaN,JUAN PEDRO SABA
130425,24233,47840,Director,2018/03/12,NaN,MÓNICA SUSANA RAMÍREZ
130426,20420,82885,Director,2015/08/14,NaN,ALBERTO EDGARDO BARBIERI
130427,20422,91416,Director,2015/08/14,NaN,JOSE LUIS POLTI


## Se trabaja con el df __proyDisciplina__

El df 'proyDisciplina' se relaciona con la el df 'disciplina' mediante el 'dsiciplina_id'. 
 - Se agrega al df 'proyDisciplina' las columnas que nos interesan del df 'disciplina' para reducir una relación.

In [27]:
#Se cambia los valores 0 por -1 porque en el df 'disciplina' esos valores están sin espeficiar. Y vamos a considera que el valor 0
#cumple esa fucnción de no estar espeficado
proyDisciplina['disciplina_id'] = proyDisciplina['disciplina_id'].replace(0, -1)

In [28]:
proyDisciplina = pd.merge(proyDisciplina,
             disciplina[['disciplina_id', 'gran_area_descripcion', 'area_descripcion', 'disciplina_descripcion']],
             on='disciplina_id',
             how='left'
)

In [29]:
proyDisciplina.shape

(19573, 5)

In [30]:
proyDisciplina

,proyecto_id,disciplina_id,gran_area_descripcion,area_descripcion,disciplina_descripcion
0,117,1,CIENCIAS NATURALES Y EXACTAS,SIN DATOS,SIN DATOS
1,128,53,INGENIERÍAS Y TECNOLOGÍAS,SIN DATOS,SIN DATOS
2,133,1,CIENCIAS NATURALES Y EXACTAS,SIN DATOS,SIN DATOS
3,134,174,CIENCIAS AGRÍCOLAS,SIN DATOS,SIN DATOS
4,138,108,CIENCIAS MÉDICAS Y DE LA SALUD,SIN DATOS,SIN DATOS
...,...,...,...,...,...
19568,24651,249,CIENCIAS AGRÍCOLAS,Ciencias Veterinarias,Ciencias Veterinarias
19569,24652,36,CIENCIAS NATURALES Y EXACTAS,Ciencias Biológicas,"Biología Celular, Microbiología"
19570,24652,107,INGENIERÍAS Y TECNOLOGÍAS,Otras Ingenierías y Tecnologías,Otras Ingenierías y Tecnologías
19571,24653,36,CIENCIAS NATURALES Y EXACTAS,Ciencias Biológicas,"Biología Celular, Microbiología"


## Se trabaja con el df __proyBeneficiario__
  - el df 'proyBeneficiario' se relaciona con el df 'organizaciones' por la columna 'organización_id'
    - Dentro 'oganizaciones' se relaciona con el df 'instituciones' por la columna 'tipo_institucion_id'
        - Se van a cambiar los valores de la columna para obtener los nombres y reducir una relación
    - Dentro de 'organizaciones' se relaciona con el df 'tipoEntidad' por la columna 'tipo_entidad_privada_id'
        - Se van a cambiar los valores de la columna para obtener los nombres y reducir una relación
    - Dentro de 'organizaciones' se relaciona con el df 'estadoOrganizacion' por la columna 'estado_organizacion'
        - Se van a cambiar los valores de la columna para obtener los nombres y reducir una relación

In [31]:
##Se crea un diccionario para el df instituciones
dictTipoInstitucion = {}

for index, row in tipoInstitucion.iterrows():
    dictTipoInstitucion[row['tipo_institucion_id']] = row['tipo_institucion_desc']

In [32]:
#Para el df organizaciones se mapea la columna y se reemplaza con los valores del diccionario. Los valores que no estén en el diccionario quedaran como 
# 'nan' que luego se van a reemplazar por el texto 'Sin especificar'
organizaciones['tipo_institucion_id'] = organizaciones['tipo_institucion_id'].map(dictTipoInstitucion)
organizaciones['tipo_institucion_id'] = organizaciones['tipo_institucion_id'].fillna('Sin especificar')

In [33]:
#Se descartan columna que no se van a utilzar
organizaciones = organizaciones.drop(columns={
    'institucion_nivel1_id',
    'institucion_nivel2_id',
    'institucion_nivel3_id',
    'institucion_nivel4_id',
    'institucion_nivel5_id',
    'institucion_nivel6_id'
})

In [34]:
organizaciones.head()

,organizacion_id,tipo_institucion_id,institucion_nivel1_descripcion,institucion_nivel2_descripcion,institucion_nivel3_descripcion,institucion_nivel4_descripcion,institucion_nivel5_descripcion,institucion_nivel6_descripcion,sigla,nivel,tipo_entidad_privada_id,estado_organizacion_id,es_doble_dependencia
0,1,Entidad administrativa de gobierno,PRESIDENCIA DE LA NACION,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,3,N
1,2,Entidad administrativa de gobierno,MINISTERIO DE DEFENSA,NaN,NaN,NaN,NaN,NaN,MINDEF,1,NaN,3,N
2,3,Entidad administrativa de gobierno,MINISTERIO DE HACIENDA,NaN,NaN,NaN,NaN,NaN,MIN HACIENDA,1,NaN,3,N
3,4,Entidad administrativa de gobierno,MINISTERIO DEL INTERIOR,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,3,N
4,5,Entidad administrativa de gobierno,MINISTERIO DE RELACIONES EXTERIORES Y CULTO,NaN,NaN,NaN,NaN,NaN,MIN RELACIONES EXT NACION,1,NaN,3,N


In [35]:
#Se cambian los valores 'nan' por 0 para poder luego cambiar el tipo de valor a 'int64' y que coincida con la columna del otro df
organizaciones['tipo_entidad_privada_id'] = organizaciones['tipo_entidad_privada_id'].fillna(0)

In [36]:
#Se cambia el tipo de dato a 'int64' para poder reemplazar los valores según esa columna
organizaciones['tipo_entidad_privada_id'] = organizaciones['tipo_entidad_privada_id'].astype('int64')

In [37]:
##Se crea un diccionario para el df tipoEntidad
dictTipoEntidad = {}

for index, row in tipoEntidad.iterrows():
    dictTipoEntidad[row['id']] = row['descripcion']

In [38]:
#Para el df tipoEntidad se mapea la columna y se reemplaza con los valores del diccionario. Los valores que no estén en el diccionario quedaran como 
# 'nan' que luego se van a reemplazar por el texto 'Sin especificar'
organizaciones['tipo_entidad_privada_id'] = organizaciones['tipo_entidad_privada_id'].map(dictTipoEntidad)
organizaciones['tipo_entidad_privada_id'] = organizaciones['tipo_entidad_privada_id'].fillna('Sin especificar')

In [39]:
organizaciones.head()

,organizacion_id,tipo_institucion_id,institucion_nivel1_descripcion,institucion_nivel2_descripcion,institucion_nivel3_descripcion,institucion_nivel4_descripcion,institucion_nivel5_descripcion,institucion_nivel6_descripcion,sigla,nivel,tipo_entidad_privada_id,estado_organizacion_id,es_doble_dependencia
0,1,Entidad administrativa de gobierno,PRESIDENCIA DE LA NACION,NaN,NaN,NaN,NaN,NaN,NaN,1,Sin especificar,3,N
1,2,Entidad administrativa de gobierno,MINISTERIO DE DEFENSA,NaN,NaN,NaN,NaN,NaN,MINDEF,1,Sin especificar,3,N
2,3,Entidad administrativa de gobierno,MINISTERIO DE HACIENDA,NaN,NaN,NaN,NaN,NaN,MIN HACIENDA,1,Sin especificar,3,N
3,4,Entidad administrativa de gobierno,MINISTERIO DEL INTERIOR,NaN,NaN,NaN,NaN,NaN,NaN,1,Sin especificar,3,N
4,5,Entidad administrativa de gobierno,MINISTERIO DE RELACIONES EXTERIORES Y CULTO,NaN,NaN,NaN,NaN,NaN,MIN RELACIONES EXT NACION,1,Sin especificar,3,N


In [40]:
#Se crea un diccionario para el df estadoOrganizacion
dictEstadoOrganizacion = {}

for index, row in estadoOrganizacion.iterrows():
    dictEstadoOrganizacion[row['id']] = row['estado_organizacion']

In [41]:
#Para el df tipoEntidad se mapea la columna y se reemplaza con los valores del diccionario. Los valores que no estén en el diccionario quedaran como 
# 'nan' que luego se van a reemplazar por el texto 'Sin especificar'
organizaciones['estado_organizacion_id'] = organizaciones['estado_organizacion_id'].map(dictEstadoOrganizacion)

In [42]:
df.head()

,proyecto_id,proyecto_fuente,titulo,fecha_inicio,fecha_finalizacion,resumen,moneda_id,monto_total_solicitado,monto_total_adjudicado,monto_financiado_solicitado,monto_financiado_adjudicado,tipo_proyecto_id,codigo_identificacion,palabras_clave,estado_id,fondo_anpcyt,cantidad_miembros_F,cantidad_miembros_M,sexo_director
0,8051,ANPCYT,Desarrollo a escala piloto de liposomas de ver...,2008/11/05,2012/12/21,No publicable,Pesos,384996.50,464959.37,190318.85,190318.65,6,ANR 600 CB 001/06,NaN,Finalizado,FONTAR,NaN,NaN,M
1,8052,ANPCYT,Desarrollo de tecnología de cultivo de células...,2008/11/12,2012/12/28,No publicable,Pesos,663677.72,623177.72,329400.00,300900.00,6,ANR 600 NA 352/06,NaN,Finalizado,FONTAR,NaN,NaN,F
2,8053,ANPCYT,Desarrollo de un Sistema de Administración Tra...,2008/01/10,2012/07/27,No publicable,Pesos,1176264.00,924564.00,583830.00,392730.00,6,ANR 600 NA 208/06,NaN,Finalizado,FONTAR,NaN,NaN,M
3,8061,ANPCYT,Desarrollo de un sistema de geoposicionamiento...,2008/09/24,2012/07/03,No publicable,Pesos,241752.00,241752.00,117500.00,117500.00,6,ANR 600 CO 002/06,NaN,Finalizado,FONTAR,NaN,NaN,M
4,8066,ANPCYT,Desarrollo de una plantación modelo de una nue...,2008/05/23,2011/04/01,No publicable,Pesos,193600.00,177100.00,91800.00,83800.00,6,ANR 600 NA 029/06,NaN,Finalizado,FONTAR,NaN,NaN,M


In [43]:
proyBeneficiario.head()

,proyecto_id,organizacion_id,persona_id,financiadora,ejecutora,evaluadora,adoptante,beneficiaria,adquiriente,porcentaje_financiamiento
0,1,5391.0,NaN,Y,N,Y,N,N,N,60.0
1,1,64.0,NaN,Y,Y,N,N,N,N,40.0
2,3,422.0,NaN,Y,Y,N,N,N,N,10.0
3,3,5391.0,NaN,Y,N,Y,N,N,N,80.0
4,3,326.0,NaN,Y,Y,N,N,N,N,10.0


In [44]:
organizaciones.head()

,organizacion_id,tipo_institucion_id,institucion_nivel1_descripcion,institucion_nivel2_descripcion,institucion_nivel3_descripcion,institucion_nivel4_descripcion,institucion_nivel5_descripcion,institucion_nivel6_descripcion,sigla,nivel,tipo_entidad_privada_id,estado_organizacion_id,es_doble_dependencia
0,1,Entidad administrativa de gobierno,PRESIDENCIA DE LA NACION,NaN,NaN,NaN,NaN,NaN,NaN,1,Sin especificar,ALTA,N
1,2,Entidad administrativa de gobierno,MINISTERIO DE DEFENSA,NaN,NaN,NaN,NaN,NaN,MINDEF,1,Sin especificar,ALTA,N
2,3,Entidad administrativa de gobierno,MINISTERIO DE HACIENDA,NaN,NaN,NaN,NaN,NaN,MIN HACIENDA,1,Sin especificar,ALTA,N
3,4,Entidad administrativa de gobierno,MINISTERIO DEL INTERIOR,NaN,NaN,NaN,NaN,NaN,NaN,1,Sin especificar,ALTA,N
4,5,Entidad administrativa de gobierno,MINISTERIO DE RELACIONES EXTERIORES Y CULTO,NaN,NaN,NaN,NaN,NaN,MIN RELACIONES EXT NACION,1,Sin especificar,ALTA,N


In [45]:
df.shape

(19266, 19)

In [46]:
proyParticipante.shape

(130429, 6)

In [47]:
proyDisciplina.shape

(19573, 5)

In [48]:
proyBeneficiario.shape

(39467, 10)

In [49]:
organizaciones.shape

(18912, 13)

In [50]:
df['fecha_finalizacion'] = df['fecha_finalizacion'].fillna('Sin finalizar')

In [51]:
listado = df.columns

El siguiente bloque de código se comenta para que no genere los archivos y corra un error

In [52]:
#Al momento de exportar el archivo df se genera un problema por un caracter no soportado en excel. 
#Se analizan las columnas individualmente para identificar cual es la columna que genera el problema
# listado = df.columns
# for i in listado:
#    print(f'Vamos a imprimir {i}')
#    df[[f'{i}']].to_excel(f'{i}.xlsx', index=False)
#    print('Se imprimio correctamente')

In [53]:
#Una vez identificada la columna que contiene el caracter que genera el problema se lo reemplaza para que no genere errores.
df['titulo'] = df['titulo'].str.replace('\x0B', '')

In [54]:
proyParticipante.to_excel('archivos_finales/proyParticipante.xlsx', index=False)

In [55]:
proyDisciplina.to_excel('archivos_finales/proyDisciplina.xlsx', index=False)

In [56]:
proyBeneficiario.to_excel('archivos_finales/proyBeneficiario.xlsx', index=False)

In [57]:
organizaciones.to_excel('archivos_finales/organizaciones.xlsx', index=False)

In [58]:
df.to_excel('archivos_finales/proyectosGenerales.xlsx', index=False)